In [2]:
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from imutils import paths

from subprocess import check_output

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.densenet import preprocess_input


C:\Users\jdu12\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# load a single image to np array
def get_image(img_path):
    img = load_img(img_path, target_size=(299, 299))
    img = img_to_array(img)
    return img

In [ ]:
# loading train images
trainPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/train/'))
print(len(trainPaths))
train = np.array([get_image(img_path) for img_path in trainPaths])
print(train.shape)

In [4]:
def to_one_hot(labels):
    one_hot_labels = []
    id2class = dict()
    class2id = dict()      ##!!! class(integer) to id(whale class)
    counter = 0
    for id in labels:
        if id not in id2class.keys():
            one_hot_labels.append(counter)
            id2class[id] = counter
            class2id[counter] = id
            counter += 1
        else:
            one_hot_labels.append(id2class[id])
    one_hot_labels = to_categorical(one_hot_labels, num_classes = 4251)
    return one_hot_labels, class2id, id2class

In [10]:
df = pd.read_csv("C:/Users/jdu12/Desktop/humpback/train.csv")

labels = df['Id']
labels, class2id, id2class = to_one_hot(labels)
#print(labels[0:10])

In [6]:
# split the data into train and val
# The dataset is super unbalanced, as there are many classes that contains only 1 image
# As a result, train/val data cannot be split before generating more data by augmentation


In [7]:
# define data generator
#use of an image generator for preprocessing and data augmentation
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

In [8]:
# acquire the model
model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=True, weights=None, classes=4251)

In [12]:
batch_size = 32
epochs = 180

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = 'C:/Users/jdu12/Desktop/humpback/saved_model/InceptionResnetV2/'
routine_dir = model_dir + "routine-{epoch:02d}-{acc:.2f}.hdf5"
routine_save = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{acc:.2f}.hdf5"
best_save = ModelCheckpoint(best_dir, monitor='acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=3)


model.fit_generator(image_gen.flow(train, labels, batch_size=batch_size),
          steps_per_epoch=  train.shape[0]//batch_size,
          epochs=epochs, verbose=1, callbacks=[routine_save, best_save])
          #class_weight=class_weight_dic)



Epoch 1/180
307/307 [==============================] - 274s 893ms/step - loss: 8.2720 - acc: 0.0797
Epoch 2/180
307/307 [==============================] - 237s 773ms/step - loss: 7.7850 - acc: 0.0817
Epoch 3/180
307/307 [==============================] - 223s 727ms/step - loss: 7.4928 - acc: 0.0827
Epoch 4/180
307/307 [==============================] - 223s 726ms/step - loss: 7.1793 - acc: 0.0821
Epoch 5/180
307/307 [==============================] - 223s 728ms/step - loss: 6.8994 - acc: 0.0819
Epoch 6/180
307/307 [==============================] - 223s 727ms/step - loss: 6.6493 - acc: 0.0827
Epoch 7/180
307/307 [==============================] - 223s 728ms/step - loss: 6.4575 - acc: 0.0819
Epoch 8/180
307/307 [==============================] - 223s 727ms/step - loss: 6.2710 - acc: 0.0818
Epoch 9/180
307/307 [==============================] - 223s 728ms/step - loss: 6.1134 - acc: 0.0823
Epoch 10/180
307/307 [==============================] - 223s 728ms/step - loss: 6.0554 - acc: 0.0804

307/307 [==============================] - 223s 728ms/step - loss: 0.1408 - acc: 0.9583
Epoch 83/180
307/307 [==============================] - 223s 727ms/step - loss: 0.1470 - acc: 0.9554
Epoch 84/180
307/307 [==============================] - 223s 727ms/step - loss: 0.1524 - acc: 0.9561
Epoch 85/180
307/307 [==============================] - 224s 728ms/step - loss: 0.1418 - acc: 0.9581
Epoch 86/180
307/307 [==============================] - 223s 727ms/step - loss: 0.1542 - acc: 0.9530
Epoch 87/180
307/307 [==============================] - 223s 727ms/step - loss: 0.1421 - acc: 0.9554
Epoch 88/180
307/307 [==============================] - 223s 728ms/step - loss: 0.1232 - acc: 0.9609
Epoch 89/180
307/307 [==============================] - 223s 727ms/step - loss: 0.1403 - acc: 0.9576
Epoch 90/180
307/307 [==============================] - 224s 728ms/step - loss: 0.1182 - acc: 0.9619
Epoch 91/180
307/307 [==============================] - 223s 728ms/step - loss: 0.1211 - acc: 0.9628
Epo

307/307 [==============================] - 223s 727ms/step - loss: 0.0789 - acc: 0.9664
Epoch 163/180
307/307 [==============================] - 223s 727ms/step - loss: 0.0757 - acc: 0.9665
Epoch 164/180
307/307 [==============================] - 223s 727ms/step - loss: 0.0632 - acc: 0.9696
Epoch 165/180
307/307 [==============================] - 223s 727ms/step - loss: 0.0546 - acc: 0.9716
Epoch 166/180
307/307 [==============================] - 223s 728ms/step - loss: 0.0559 - acc: 0.9733
Epoch 167/180
307/307 [==============================] - 223s 727ms/step - loss: 0.0653 - acc: 0.9680
Epoch 168/180
307/307 [==============================] - 235s 765ms/step - loss: 0.0630 - acc: 0.9709
Epoch 169/180
307/307 [==============================] - 240s 783ms/step - loss: 0.0582 - acc: 0.9714
Epoch 170/180
307/307 [==============================] - 240s 781ms/step - loss: 0.0567 - acc: 0.9706
Epoch 171/180
307/307 [==============================] - 240s 780ms/step - loss: 0.0670 - acc: 0

In [5]:
# loading test images
testPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/test/'))
print(len(testPaths))
test_images = np.array([get_image(img_path) for img_path in testPaths])/255
print(test_images.shape)

15610
(15610, 299, 299, 3)


In [16]:
# loading trained model
model = load_model('C:/Users/jdu12/Desktop/humpback/saved_model/InceptionResnetV2/' + 'routine-80-0.95.hdf5')

In [17]:
# predict!
pred = model.predict(test_images)

In [18]:
# testing and generating submission file
import warnings
from os.path import split

with open("incepResV2_submission.csv","w") as f:
    f.write("Image,Id\n")
    top_5 = np.argsort(pred)[:,-1:-6:-1]   # get the top 5 most likely classes
    for i in range(top_5.shape[0]):
        cur_tags = ''
        cur_image_name = testPaths[i].split('/')[-1]
        for j in range(5):
            cur_tags = cur_tags + ' ' + class2id[top_5[i][j]]
        f.write("%s,%s\n" %(cur_image_name, cur_tags))     

In [52]:
top_5 = np.argsort(pred)[:,-1:-6:-1]
print(top_5.shape)
a = np.asarray([[1, 2, 3, 4, 5, 6, 7, 8],[1, 2, 3, 4, 5, 6, 7, 8]])
print(a[:,-1:-6:-1])


[[8 7 6 5 4]
 [8 7 6 5 4]]
